In [1]:
import pandas as pd
import sklearn 
import numpy as np
from numpy import asarray, save, load, savetxt, loadtxt
import difflib
import h5py
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder #, OneHotEncoder

from functions import *

In [2]:
#Não esquecer de explicar isso de forma mais detalhada 
'''Não existe uma maneira segura de agrupar os dados de um conjunto de strings, a não ser decisões mais complexas como uso de 
uma NLP. K-means é impraticável porque o dado não é categório. A única alternativa é lidar com regiões planares, 
fazendo expansão matricial e relacionando uma string com a outra, uma por uma. Aqui usaremos uma estratégia menos custosa 
computacionalmente'''

'Não existe uma maneira segura de agrupar os dados de um conjunto de strings, a não ser decisões mais complexas como uso de \numa NLP. K-means é impraticável porque o dado não é categório. A única alternativa é lidar com regiões planares, \nfazendo expansão matricial e relacionando uma string com a outra, uma por uma. Aqui usaremos uma estratégia menos custosa \ncomputacionalmente'

In [3]:
df = dataframe_train_val()
df_test = dataframe_test()

In [4]:
df.head(len(df))

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37995,13230578,1756482,mochila personalizada galinha pintadinha,1,2,Mochila Galinha Pintadinha M,primaria 2019 1 aninho abdulzinho mochilas ani...,2016-09-17 10:49:39,18.790000,149.0,1,27,321,NaN,Lembrancinhas
37996,6736914,9301388,tag dia dos pais,1,32,30 TAGS DIA DOS PAIS 005,dia pais,2019-06-12 17:03:52,31.680000,7.0,1,1,43,NaN,Papel e Cia
37997,11017911,8732362,kit bolsa maternidade,5,31,Kit bolsa bebê maternidade personalizada,paula carvalho bebe,2018-08-24 11:43:00,543.170000,3006.0,1,4,515,18.0,Bebê
37998,6807331,1869417,festa 15 anos,1,8,Chaveiro Almofada 15 anos,yasmin centro mesa compras 15 anos 2020 lembra...,2017-10-21 18:49:56,10.720000,18.0,1,53,2456,138.0,Lembrancinhas


In [5]:
df_test.head(len(df_test))

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,4973768,2484177,lembrancinhas de 15 anos,5,8,Lembrancinha 15 anos - Lembrancinha de 15 anos,15 anos,2019-02-02 10:28:57,14.180000,10.0,1,13,91,NaN,Lembrancinhas
1,14180109,3191400,prateleira,1,32,Trio de Nichos Prateleira,prateleiras decoracao gaveteiros nichos pratel...,2017-08-23 22:17:01,97.060005,7492.0,1,1,729,32.0,Decoração
2,5058490,1414443,lembrancinhas vingadores,2,19,Caixa meia bala Vingadores,vingadores super herois,2017-05-28 04:26:03,11.580000,0.0,1,13,92,NaN,Lembrancinhas
3,576188,9188811,sacolinha galinha pintadinha,1,6,Sacolinha Galinha Pintadinha,sacolinhas galinha pintadinha sacolinhas perso...,2017-02-14 10:19:28,13.390000,8.0,1,15,34,NaN,Lembrancinhas
4,5190417,2497208,vaso de barro atacado,1,23,30 Mini Vaso 6cm Cerâmica Lembrancinha casamen...,cha bebe lhama mini vasos c adesivos,2018-10-19 00:22:33,11.470000,2009.0,1,39,741,NaN,Lembrancinhas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,726319,4527735,lembrancinhas de igreja dia dos pais,2,14,Dia dos pais,dia pais,2016-07-09 22:06:53,15.770000,8.0,1,10,279,NaN,Lembrancinhas
496,5702671,2857393,porta fraldas,1,34,Porta fraldas para bolsa,porta fraldas bebe porta fraldas portatil kit ...,2016-05-29 11:02:11,59.880000,6.0,1,1,248,NaN,Bebê
497,11130757,9865080,espelho adnet,1,9,Espelho Adnet 40 cm - Marrom com Alça Caramelo,espelho sala decorar decoracao,2018-09-06 11:38:39,258.520000,2000.0,0,6,947,141.0,Decoração
498,2739553,7358636,polaina tricot,1,0,Polainas em Tricô,polainas,2013-07-15 16:58:47,55.870000,0.0,0,1,176,NaN,Bebê


In [6]:
restricted_columns_df = get_columns(df, ['title', 'category'])
restricted_columns_df_test = get_columns(df_test, ['title', 'category'])

In [7]:
restricted_columns_df.head(len(df))

,title,category
0,Mandala Espírito Santo,Decoração
1,Cartão de Visita,Papel e Cia
2,Organizador expositor p/ 70 esmaltes,Outros
3,Jogo de Lençol Berço Estampado,Bebê
4,ADESIVO BOX DE BANHEIRO,Decoração
...,...,...
37995,Mochila Galinha Pintadinha M,Lembrancinhas
37996,30 TAGS DIA DOS PAIS 005,Papel e Cia
37997,Kit bolsa bebê maternidade personalizada,Bebê
37998,Chaveiro Almofada 15 anos,Lembrancinhas


In [8]:
X = restricted_columns_df.iloc[:, 0:1].values
Y = restricted_columns_df.iloc[:, 1].values
Xtest = restricted_columns_df_test.iloc[:, 0:1].values
Ytest = restricted_columns_df_test.iloc[:,1].values

In [9]:
########## Organizing dataset #############

In [10]:
#############def calc_pairwise_similarity(lista):##################                                                                                                                                                                         

In [11]:
#To teste calc_pairwise_similarity
str_titles = X.T[0].tolist()
pairwise_similarity = calc_pairwise_similarity(str_titles)

In [12]:
#to assert output
#(0, 37824)	0.2607881353491155
print(pairwise_similarity[0,37824])

0.2607881353491155


In [13]:
############def max_similarity(pairwise_similarity):##############


In [14]:
title_similarity = max_similarity(pairwise_similarity)   

We've already made 0 similaritys calculations.
We've already made 1000 similaritys calculations.
We've already made 2000 similaritys calculations.
We've already made 3000 similaritys calculations.
We've already made 4000 similaritys calculations.
We've already made 5000 similaritys calculations.
We've already made 6000 similaritys calculations.
We've already made 7000 similaritys calculations.
We've already made 8000 similaritys calculations.
We've already made 9000 similaritys calculations.
We've already made 10000 similaritys calculations.
We've already made 11000 similaritys calculations.
We've already made 12000 similaritys calculations.
We've already made 13000 similaritys calculations.
We've already made 14000 similaritys calculations.
We've already made 15000 similaritys calculations.
We've already made 16000 similaritys calculations.
We've already made 17000 similaritys calculations.
We've already made 18000 similaritys calculations.
We've already made 19000 similaritys calcula

In [15]:
######### Tramsform X_test in X_test_labelled##############

In [16]:
X_train = np.array(title_similarity).reshape(38000,1)
X_train_str = decoder_label_to_unique(X_train,X)
print(X_train_str)

['Mandala Espírito Santo', 'Cartão de Visita', 'Organizador expositor p/ 70 esmaltes', 'Jogo de Lençol Berço Estampado', 'ADESIVO BOX DE BANHEIRO', 'Álbum de figurinhas dia dos pais', 'Arranjo de Flores - Orquidias', 'Kit Aromarizador + sacola / Lembrancinha Maternidade', 'Álbum de figurinhas dia dos pais', 'Manta para bebê personalizada de Nuvem com nome', 'Crachá + Cordão Personalizado + Porta Crachá + Arte Grátis', 'Álbum de figurinhas dia dos pais', 'Mini Tabua de Carne Lembrancinhas De Casamento/chá De Panela', 'Criado Mudo Espelhado com 3 gavetas 70x35x55', 'Chinelo Dia dos Pais', 'LOCAÇÃO - Muro inglês (Painel de Folhas)', 'Lembrancinha de batizado', 'chaveiro dia dos pais', 'Álbum de figurinhas dia dos pais', 'Nicho Garden - Kit 3 Peças', 'Lembrancinha de batizado', 'Berloque', 'Naninha Para Bebê', 'Planner financeiro - A6', 'Manta para bebê personalizada de Nuvem com nome', 'LEMBRANCINHA 15 ANOS', 'Sacolinha com Difusor', 'Caixinha para lembrancinha de maternidade menina', 'ch

In [17]:
#########Labelled###########

In [18]:
X_test = variabletest_to_labelled(X_train,X_train_str,Xtest)

We've already made 0 similaritys calculations.
We've already made 10 similaritys calculations.
We've already made 20 similaritys calculations.
We've already made 30 similaritys calculations.
We've already made 40 similaritys calculations.
We've already made 50 similaritys calculations.
We've already made 60 similaritys calculations.
We've already made 70 similaritys calculations.
We've already made 80 similaritys calculations.
We've already made 90 similaritys calculations.
We've already made 100 similaritys calculations.
We've already made 110 similaritys calculations.
We've already made 120 similaritys calculations.
We've already made 130 similaritys calculations.
We've already made 140 similaritys calculations.
We've already made 150 similaritys calculations.
We've already made 160 similaritys calculations.
We've already made 170 similaritys calculations.
We've already made 180 similaritys calculations.
We've already made 190 similaritys calculations.
We've already made 200 similari

In [19]:
X_test = np.array(X_test).reshape(-1,1)

In [20]:
X_concatenate = np.concatenate((X_train,X_test))
Y_concatenate = np.concatenate((Y,Ytest))

In [21]:
labelencoder_X = LabelEncoder()
X_concatenate_labelled = labelencoder_X.fit_transform(X_concatenate)
labelencoder_Y = LabelEncoder()
Y_concatenate_labelled = labelencoder_Y.fit_transform(Y_concatenate)

/usr/local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [22]:
onehotencoderX = OneHotEncoder()
X_onehot = onehotencoderX.fit_transform(X_concatenate_labelled.reshape(-1, 1)).toarray()
onehotencoderY = OneHotEncoder()
Y_onehot = onehotencoderY.fit_transform(Y_concatenate_labelled.reshape(-1, 1)).toarray()

In [23]:
print(X_onehot.shape,Y_onehot.shape)

(38500, 2049) (38500, 6)


In [24]:
X_train__ = X_onehot[0:38000]
X_test__ = X_onehot[38000:38500]
Y_train__ = Y_onehot[0:38000]
Y_test__ = Y_onehot[38000:38500]

In [25]:
print(X_train__.shape,X_test__.shape,Y_train__.shape,Y_test__.shape)

(38000, 2049) (500, 2049) (38000, 6) (500, 6)


In [26]:
print(type(X_train__),X_train__.shape)
print(type(X_test__),X_test__.shape)
print(type(Y_train__),Y_train__.shape)
print(type(Y_test__),Y_test__.shape)
print(type(X),X.shape)
print(type(Y),Y.shape)
print(type(Xtest),Xtest.shape)
print(type(Ytest),Ytest.shape)

<class 'numpy.ndarray'> (38000, 2049)
<class 'numpy.ndarray'> (500, 2049)
<class 'numpy.ndarray'> (38000, 6)
<class 'numpy.ndarray'> (500, 6)
<class 'numpy.ndarray'> (38000, 1)
<class 'numpy.ndarray'> (38000,)
<class 'numpy.ndarray'> (500, 1)
<class 'numpy.ndarray'> (500,)


In [27]:
#Saving in a h5 file

In [28]:


hf = h5py.File(os.environ['PREPROCESSED_DATA'],'w')

X_train = hf.create_group('X_train')
Y_train = hf.create_group('Y_train')
X_test = hf.create_group('X_test')
Y_test = hf.create_group('Y_test')

X_train.create_dataset("X_train", data=X_train__)
Y_train.create_dataset("Y_train", data=Y_train__)
X_test.create_dataset("X_test", data=X_test__)
Y_test.create_dataset("Y_test", data=Y_test__)



hf.close()



In [ ]:
'''
Field	Description
product_id	Product numeric identification #pas besoin
seller_id	Seller numeric identification #pas besoin
query	The text inserted by users #ok
search_page	The page number the product appeared (min 1 and max 5) #pas besoin
position	The position the product appeared in the search page (min 0 and max 38) #pas besoin
title	Product title #ok 
concatenated_tags	Product tags inserted by the seller #ok
creation_date	The date of product registration in Elo7 platform #pas besoin
price	The product price (R$) #ok
weight	The weight (grams) of a product unit #ok
express_delivery	Indicates if the product has already been made (1) or not (0) #pas besoin
minimum_quantity	The minimum quantity the seller sells the product #pas besoin
view_counts	The number of times the product was clicked in the last three months #pas besoin 
order_counts	The number of times the product was purchased in the last three months #pas besoin 
category	Product category # ok'''

In [ ]:
'''
query	The text inserted by users #ok 3
title	Product title #ok 6
concatenated_tags	Product tags inserted by the seller #ok 7 
price	The product price (R$) #ok 9 
weight	The weight (grams) of a product unit #ok 10
category	Product category # ok 15'''